In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df =pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
#Preprocessing

#Embarked have 2 , very few missing values - replace with mode
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])



#extract deck from first lette of cabin 

df['Deck'] = df['Cabin'].str[0]





In [ ]:
df.isnull().sum()

In [ ]:
#handling missing age

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
features_for_age = ['Age','Fare','Pclass','SibSp','Parch']

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[features_for_age])

# Impute
imputer = KNNImputer(n_neighbors=5)
imputed_scaled = imputer.fit_transform(df_scaled)

# Inverse scale to original space
df_imputed = scaler.inverse_transform(imputed_scaled)

# Update only 'Age' in original df
df['Age'] = df_imputed[:, features_for_age.index('Age')]

In [ ]:

df.isnull().sum()


In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df['Cabin'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
for col in ['Sex','Embarked']:
    
        plt.figure(figsize=(8,4))
        sns.barplot(x=col,y='Survived',data = df)
        plt.title(f'Barplot of {col}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show

In [ ]:
for col in df.select_dtypes(include=['int64','float64']).columns:
    if col != 'Survived':

       
        sns.boxplot(x='Survived',y=col,data=df)
        plt.tight_layout()
        plt.show()

In [ ]:
for col in df.select_dtypes(include=['int64','float64']).columns:
    if col != 'Survived':

       
        sns.violinplot(x='Survived',y=col,data=df)
        plt.tight_layout()
        plt.show()

In [ ]:
#extracting title from name

df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.',expand=False)


In [ ]:
df.columns

In [ ]:
#use classifier to find missing vallues in deck (for cabin)

train_deck = df[df['Deck'].notnull()].copy()

test_deck = df[df['Deck'].isnull()].copy()

In [ ]:
#encoding categorical values into numerical

from sklearn.tree import DecisionTreeClassifier

train_deck['Title'] = train_deck['Title'].astype('category').cat.codes
train_deck['Sex'] = train_deck['Sex'].astype('category').cat.codes
train_deck['Embarked'] = train_deck['Embarked'].astype('category').cat.codes




In [ ]:
#training a classifier for deck predicition using Pclass, SibSp,Sex,Parch, Title

features_for_deck = ['Pclass','SibSp','Sex','Parch','Title']
X_train = train_deck[features_for_deck]
y_train = train_deck['Deck']

model = DecisionTreeClassifier(max_depth=4)

model.fit(X_train,y_train)

In [ ]:
test_deck['Title'] = test_deck['Title'].astype('category').cat.codes
test_deck['Sex'] = test_deck['Sex'].astype('category').cat.codes
test_deck['Embarked'] = test_deck['Embarked'].astype('category').cat.codes

In [ ]:
#predicting deck for unknown decks

X_test = test_deck[features_for_deck]

y_pred = model.predict(X_test)

In [ ]:
#replacing original dataframe null values with predicted deck values
df.loc[df['Deck'].isnull(),'Deck'] = y_pred


In [ ]:
df.isnull().sum()

In [ ]:
#drop cabin as we no longer need it

df.drop('Cabin',axis=1,inplace=True)

In [ ]:
df

In [ ]:
df['Deck'].value_counts()

In [ ]:
#drop name
df.drop(['Name'], axis=1,inplace=True)


In [ ]:
df.head()

In [ ]:
#categoricla encoding
df['Sex'] = df['Sex'].map({'male':0,'female':1})

In [ ]:
df['Embarked'] = df['Embarked'].map({'S':0,'C':1,'Q':2})

In [ ]:
df['Deck'] = df['Deck'].astype('category').cat.codes


In [ ]:
df.head()

In [ ]:
df['Title'].value_counts()

In [ ]:
#replacing titles meaningfully

df['Title'] = df['Title'].replace(['Mlle','Ms'],'Miss')
df['Title'] = df['Title'].replace('Mme','Mrs')

df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [ ]:
df['Title'].value_counts()

In [ ]:
df['Title'] = df['Title'].astype('category').cat.codes

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB



In [ ]:
df.drop(['Ticket','PassengerId'],axis=1,inplace=True)

In [ ]:
X = df.drop('Survived',axis=1)
y = df['Survived'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#trying different classiication models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Naive Bayes": GaussianNB()
}

In [ ]:
result=[]
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    result.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_proba)
    })

In [ ]:
result_df = pd.DataFrame(result).sort_values(by='F1 Score', ascending=False)
print(result_df)